In [ ]:
"""
Outage Analytics Agent using LangGraph
--------------------------------------

✅ Parses user queries (like "show average outage duration per partner for last 6 months")
✅ Generates SQL dynamically using LLM
✅ Runs SQL against DuckDB
✅ Creates charts + PPTs if requested
✅ Provides natural summaries

"""

In [ ]:
import os
import json
import re
import io
import base64
import tempfile
import matplotlib.pyplot as plt
import duckdb
from pptx import Presentation
from pptx.util import Inches
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Dict, Any
from dotenv import load_dotenv 
load_dotenv(override=True)  # take environment variables from .env file

In [3]:
# ----------------------------------------------------------------------
# 1️⃣ Helper Functions
# ----------------------------------------------------------------------

In [ ]:
def column_name_prompt_template():
    prompt = """
    - Database table: outages
    - Available columns:
    - partner_name
    - outage_type
    - issue_details
    - duration_hours
    - outage_start_time
    - outage_end_time   
    - business_impact
    - email_date
    - resolution_details
    """
    return prompt

In [ ]:
def duckdb_prompt_template():
    prompt = """
    
    DuckDB SQL Rules Summary for LLM
    1. General Syntax
    - Only SELECT queries.
    - No markdown, comments, or semicolons required.
    - Use only functions available in DuckDB.

    2. Date & Time Rules
    - Valid: CURRENT_DATE, CURRENT_TIMESTAMP
    - Interval subtraction: CURRENT_DATE - INTERVAL '6' MONTH
    - Truncation: date_trunc('month', email_date)
    - Formatting: strftime(outage_start_time, '%Y-%m-%d %H:%M')
    - Extraction: extract(month FROM email_date)
    - Invalid: DATEADD, DATE_SUB, DATEDIFF, TIMESTAMPADD

    3. Timestamp Rules
    - Do not convert timestamps with /1000.
    - Use timestamps directly with strftime().

    4. Numeric Functions
    - ROUND, CEIL, FLOOR, ABS, POWER.

    5. String Functions
    - LOWER, UPPER, LENGTH, SUBSTR, CONCAT, REPLACE.

    6. Aggregation Rules
    - Aggregate with COUNT, SUM, AVG, MIN, MAX.
    - All non-aggregated columns must be in GROUP BY.

    7. ORDER BY Rules
    - ORDER BY partner_name ASC
    - ORDER BY COUNT(*) DESC

    8. Filtering
    - WHERE duration_hours > 1
    - WHERE partner_name ILIKE '%global%'
    - Date ranges: WHERE email_date BETWEEN '2024-01-01' AND '2024-12-31'

    9. Nonexistent Functions (Avoid)
    - DATEADD, DATE_SUB, DATEDIFF, TIMESTAMPADD
    - FORMAT_DATETIME, MONTHNAME, YEARWEEK, DAYOFWEEK

    10. Output Rules
    - Return only SQL.
    - No explanations.
    
    11. If the query involves both GROUP BY and window functions (OVER PARTITION BY),
    first aggregate inside a CTE, then apply window functions on the aggregated result.

    12. Never mix raw columns + GROUP BY + window functions on the same level.
    Always use this pattern:
    WITH t AS (
    SELECT col1, col2, SUM(...) AS agg
    FROM table
    GROUP BY col1, col2
    )
    SELECT col1, col2, agg, SUM(agg) OVER (PARTITION BY col1)
    FROM t;

    13. Use outage_start_time and outage_end_time for any time-based calculations.
    14. Business impact is a text field; do not perform numeric operations on it.
    15. BUSINESS IMPACT RULES:
    - The column business_impact contains long free-text descriptions (e.g., “Limited impact on order acknowledgments”).
    - business_impact is NOT numeric. Never use SUM(), AVG(), MIN(), MAX(), or any arithmetic on it.
    - business_impact should NEVER be mixed inside numeric aggregations such as SUM(duration_hours).
    - If the user asks to “summarize business impacts”, “show business impact per partner”,  
    - Business impact should be summarized separately and mentioned in a separate section
    16
    - Always qualify ambiguous columns when joining (e.g., o.partner_name, t.partner_name).
    - Do NOT use unqualified column names in JOIN queries.
    - When aggregating text:
    * Prefer LISTAGG(column, '; ') AS aggregated_column
    * STRING_AGG is allowed only if valid
    
    """
    

In [ ]:

def generate_sql(state: dict):
    """
    Takes a natural-language user query → asks LLM to generate initial SQL 
    """
    from datetime import datetime

    user_query = state.get("user_query")
    con = state.get("db_con")

    if not user_query:
        raise KeyError("Missing 'user_query' in state")
    if not con:
        raise KeyError("Missing 'db_con' in state")

    print(f"🧠 Generating SQL for user query: {user_query}")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = f"""
    You are a SQL generator for a DuckDB database.
    Your task: Write a valid DuckDB SQL SELECT statement that answers the user question below.

    User Query:
    "{user_query}"

    ### Database Schema:
    {column_name_prompt_template()}
    
    ### SQL Style Rules:
    {duckdb_prompt_template()}
            
    """
    llm_sql = llm.invoke(prompt).content.strip()
    print(f"🧮 llm SQL :\n{llm_sql}")

   
    return {"sql_query": llm_sql, "generated_at": datetime.utcnow().isoformat()}


In [5]:
def init_db(in_memory=False):
    """Initialize DuckDB database."""
    db_file = ":memory:" if in_memory else "outages.duckdb"
    con = duckdb.connect(database=db_file)
    print(f"💾 Connected to {db_file}")
    con.execute("""
        CREATE TABLE IF NOT EXISTS outages (
            partner_name VARCHAR,
            outage_type VARCHAR,
            issue_details VARCHAR,
            current_status VARCHAR,
            business_impact VARCHAR,
            manual_processing VARCHAR,
            root_cause_available VARCHAR,
            outage_start_time TIMESTAMP,
            outage_end_time TIMESTAMP,
            duration_hours DOUBLE,
            resolution_details VARCHAR,
            email_subject VARCHAR,
            email_date DATE,
            parsed_at TIMESTAMP,
            source_file VARCHAR
        );
    """)
    return con





In [6]:

def run_sql(con, sql: str):
    """Run SQL safely on DuckDB."""
    print(f"Executing SQL: {sql}")
    try:
        df = con.execute(sql).df()
        print(f"✅ SQL returned {len(df)} rows")
        return df.to_json(orient="records")
    except Exception as e:
        print(f"❌ SQL Error: {e}")
        return json.dumps({"error": str(e)})

In [7]:
# ----------------------------------------------------------------------
# 2️⃣ Universal Unwrapping + Helpers
# ----------------------------------------------------------------------

In [8]:


def unwrap_state(state):
    """Universal fix — unpacks LangGraph’s nested state safely."""
    if isinstance(state, dict) and "input" in state and isinstance(state["input"], dict):
        print("🪄 Auto-unwrapped LangGraph state.")
        merged = {**state, **state["input"]}
        merged.pop("input", None)
        return merged
    return state




In [9]:
def get_db_con(state):
    """Retrieve DB connection from state."""
    con = state.get("db_con")
    if not con:
        raise ValueError("Missing database connection in state.")
    return con

In [10]:
# ----------------------------------------------------------------------
# 3️⃣ Agent State Definition
# ----------------------------------------------------------------------

In [11]:


class AgentState(TypedDict, total=False):
    user_query: str
    sql_query: Optional[str]
    data_json: Optional[str]
    chart_uri: Optional[str]
    ppt_path: Optional[str]
    final_answer: Optional[str]
    intent: Optional[Dict[str, Any]]
    db_con: Any


In [12]:
# ----------------------------------------------------------------------
# 4️⃣ Core Nodes
# ----------------------------------------------------------------------

In [13]:


def interpret_query(state: AgentState):
    state = unwrap_state(state)
    user_query = state.get("user_query")
    if not user_query:
        raise KeyError("Missing 'user_query' in state input")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    prompt = f"""
    Determine the intent type for this user request and whether a chart or PPT is requested.
    Return a JSON object exactly, e.g. {{ "intent": "sql_query", "needs_chart": true, "notes": "..." }}
    Possible intents: sql_query, summary, report
    User query: {user_query}
    """
    resp = llm.invoke(prompt).content.strip()
    try:
        intent = json.loads(resp)
    except Exception:
        intent = {"intent": "sql_query", "needs_chart": ("chart" in user_query.lower()), "notes": ""}
    print(f"🧭 Intent: {intent}")
    return {"intent": intent, "user_query": user_query}

def execute_sql(state: AgentState):
    state = unwrap_state(state)
    sql_query = state.get("sql_query")
    con = get_db_con(state)

    data_json = run_sql(con, sql_query)
    return {"data_json": data_json}

def summarize_results(state: AgentState):
    state = unwrap_state(state)
    data_json = state.get("data_json")
    user_query = state.get("user_query")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    prompt = f"""
    Summarize the following outage analytics result for this query:
    {user_query}
    Data: {data_json[:2000]}
    Rules
    - Create a concise summary.
    - Mention only those insights that are relevant to the query.
    - If no data, say "No relevant outage data found."
    """
    summary = llm.invoke(prompt).content.strip()
    print(f"📝 Summary: {summary}")
    return {"final_answer": summary}


In [14]:

# ----------------------------------------------------------------------
# 5️⃣ Graph Definition
# ----------------------------------------------------------------------

def build_outage_agent_graph():
    graph = StateGraph(AgentState)

    graph.add_node("interpret", interpret_query)
    graph.add_node("generate_sql", generate_sql)
    graph.add_node("execute_sql", execute_sql)
    graph.add_node("summarize", summarize_results)

    graph.add_edge("interpret", "generate_sql")
    graph.add_edge("generate_sql", "execute_sql")
    graph.add_edge("execute_sql", "summarize")
    graph.set_entry_point("interpret")
    graph.set_finish_point("summarize")
    return graph.compile()


In [23]:
# ----------------------------------------------------------------------
# 6️⃣ Run Example
# ----------------------------------------------------------------------

if __name__ == "__main__":
    con = init_db(in_memory=False)
    app = build_outage_agent_graph()

    user_query = "Pull partner name, number of outags and business impacts per partner for last 3 years"

    print("\n🚀 Running outage analytics agent...\n")
    response = app.invoke({"user_query": user_query, "db_con": con})

    print("\n=== FINAL RESPONSE ===")
    print(response.get("final_answer") or "No final answer generated.")
   

💾 Connected to outages.duckdb

🚀 Running outage analytics agent...

🧭 Intent: {'intent': 'sql_query', 'needs_chart': False, 'notes': 'The user is requesting data extraction related to partners, which suggests a SQL query is needed to retrieve the information.'}
🧠 Generating SQL for user query: Pull partner name, number of outags and business impacts per partner for last 3 years
🧮 llm SQL :
WITH t AS (
    SELECT partner_name, COUNT(*) AS outage_count
    FROM outages
    WHERE outage_start_time >= CURRENT_DATE - INTERVAL '3' YEAR
    GROUP BY partner_name
)
SELECT t.partner_name, t.outage_count, 
       LISTAGG(o.business_impact, '; ') AS aggregated_business_impacts
FROM t
JOIN outages o ON t.partner_name = o.partner_name
WHERE o.outage_start_time >= CURRENT_DATE - INTERVAL '3' YEAR
GROUP BY t.partner_name, t.outage_count;
Executing SQL: WITH t AS (
    SELECT partner_name, COUNT(*) AS outage_count
    FROM outages
    WHERE outage_start_time >= CURRENT_DATE - INTERVAL '3' YEAR
    GRO